In [ ]:
# User-defined variables
work_dir = '/Users/Tem/Documents/naturalistic-threat-ptsd'
npy_dir = f"{work_dir}/data/neural-classical/masked/subcort-harvard-oxford-6mm" #directory housing masked npy arrays
mask_dir = f"{work_dir}/masks/subcort-harvard-oxford"
hemo_lag = 4
nPerms = 10000

In [ ]:
# Import packages 
import warnings
warnings.filterwarnings("ignore")
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import iqr
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import fdrcorrection

In [ ]:
# Load and define participant data
symptom_labels = ['Re-experiencing', 'Avoidance-Numbing', 'Hyperarousal']
CAPS_data = pd.read_csv(f"{work_dir}/data/subs/CAPS.csv")
cohort_IDs = pd.read_csv(f"{work_dir}/data/subs/cohort_IDs.csv")

# Load ROI labels
ROI_labels = pd.read_csv(f"{mask_dir}/ROI-labels.csv") #directory housing masks & labels

# Define n's
n_regions = len(ROI_labels)
nCohort = len(cohort_IDs)

In [ ]:
# Load event files for stimulus onsets
all_CSminus_onsets = [None] * nCohort
all_CSplus_onsets = [None] * nCohort

for i in range(nCohort):
    day1_onsets = pd.read_csv(f"{work_dir}/data/task/events-classical/{cohort_IDs['subject'][i]}.csv")
    
    day1_CSplus = day1_onsets[day1_onsets['trial_type'].str.contains('plus') & ~day1_onsets['trial_type'].str.contains('US')].reset_index()
    CSplus_onsets = [list(range(day1_CSplus['onset'][i] + hemo_lag, day1_CSplus['onset'][i] + day1_CSplus['duration'][i] + hemo_lag)) for i in range(len(day1_CSplus))]
    all_CSplus_onsets[i] = [item for sublist in CSplus_onsets for item in sublist]

    day1_CSminus = day1_onsets[day1_onsets['trial_type'].str.contains('minus')].reset_index()
    CSminus_onsets = [list(range(day1_CSminus['onset'][i] + hemo_lag, day1_CSminus['onset'][i] + day1_CSminus['duration'][i] + hemo_lag)) for i in range(len(day1_CSminus))]
    all_CSminus_onsets[i] = [item for sublist in CSminus_onsets for item in sublist]
    

In [ ]:
# Loop through ROIs to calculate interSC, classification, and correlation with symptoms using child script
counter = 0
for ROI_no in range(n_regions):
    disp_name = ROI_labels['label'][ROI_no]
    ROI_ID = ROI_labels['index'][ROI_no]
    cROI = f"HO_{ROI_ID}"
    print(f"Running Harvard-Oxford ROI #{ROI_ID}, {ROI_labels['label'][counter]}...")
    startTime = datetime.now()
    
    %run './09. run-classical-child.ipynb'
    ROI_values = pd.Series(ROI_values, index = ROI_value_labels)              
    
    if counter == 0:
        all_ROI_values = pd.DataFrame(columns = ROI_value_labels)
        all_ROI_values = all_ROI_values.append(ROI_values, ignore_index = True)
    else: 
        all_ROI_values = all_ROI_values.append(ROI_values, ignore_index = True)
        
    print(f"Runtime for ROI #{ROI_ID}: {datetime.now() - startTime}\n")    
    counter = counter + 1

In [ ]:
# Save dataframe containing all classical threat values for all ROIs 
all_ROI_values.insert(0, 'ROI', ROI_labels['label'])
date_string = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
all_ROI_values.to_csv(f"{work_dir}/data/tables/classical-multi-ROI-{nPerms}_perms-{date_string}.csv", index = False)
all_ROI_values